In [1]:
#인코더/디코더/모델 만드는 펑션은 동일
#모델 인풋아웃풋 구조도 똑같은듯
#결국 데이터 제너레이션을 하는 방법만 달라진다... ? + 피쳐 뽑고 어큐러시 계산하는 방법

In [2]:
import numpy as np
from random import randint
#from numpy import array, argmax, array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


In [18]:
a = randint(1, 50)
b = np.arange(a,a+5)
list(b)

[25, 26, 27, 28, 29]

In [16]:
b

range(46, 51)

In [46]:
def generate_sequence(length, n_unique):
    
    num = randint(1, n_unique-11)
    return list(np.arange(num, num + length))
 
def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        target = list(np.arange(source[-1],source[-1]+n_out))
#         print(source)
#         print(target)
        
        target_in = [0] + target[:-1]
#         print('target:',target)
#         print('target-1:',target[:-1])
#         print('target_in:',target_in)
        
        # encode
        src_encoded = to_categorical(source, num_classes=cardinality)
        tar_encoded = to_categorical(target, num_classes=cardinality)
        tar2_encoded = to_categorical(target_in, num_classes=cardinality)
        
        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return np.array(X1), np.array(X2), np.array(y)

def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return array(output)

In [49]:
n_features = 50 + 10
n_steps_in = 5
n_steps_out = 5
# generate a single source and target sequence
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 10000)

In [48]:
print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

(10000, 5, 60) (10000, 5, 60) (10000, 5, 60)
X1=[49, 50, 51, 52, 53], X2=[0, 53, 54, 55, 56], y=[53, 54, 55, 56, 57]


In [50]:
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [51]:
train.fit([X1, X2], y, epochs=1)

Epoch 1/1
10000/10000 [==============================] - 5s 501us/step - loss: 1.0277 - acc: 0.8188


In [52]:
total, correct = 100, 0
for _ in range(total):
    X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
    target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
    if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
    # spot check some examples
    
for _ in range(10):
    X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
    target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
    print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

NameError: name 'array' is not defined